<a href="https://colab.research.google.com/github/dmbk/Anomaly-Detection-System/blob/master/ConvLSTM_WGAN_GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install imageio
#!pip install tensorflow_datasets
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import skimage
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from PIL import Image
from tensorflow.keras import layers
import time
from tqdm import tqdm
import time
from IPython import display

from os.path import join
from os import listdir
from os.path import isfile, join, isdir

#import keras
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Add, Activation, GaussianNoise, Reshape, LeakyReLU, Conv2DTranspose, Conv3DTranspose, ConvLSTM2D, BatchNormalization, LayerNormalization, TimeDistributed, Conv2D, Conv3D, ZeroPadding3D, MaxPooling2D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model, Model
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import argparse
from os.path import dirname
import cv2
from google.colab.patches import cv2_imshow
	
from keras import backend
import tensorflow as tf
import tensorflow.keras.backend as K
keras = tf.keras
import statistics

from keras.constraints import Constraint
tf.keras.backend.set_floatx('float32')
import shutil

import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from keras import backend as K

In [0]:
class Config:
    def __init__(self, data_dir_, cwdir_name_, data_set):
        self.data_set_name = data_set
        self.data_dir = data_dir_
        self.data_set_dir = join(self.data_dir, data_set)
        self.cwdir_name = cwdir_name_
        self.cwdir = join(self.data_dir,self.cwdir_name)
        self.run_data = join(self.cwdir, "training_gan")
        self.image_dir = join(self.run_data,self.data_set_name,"Test/")
        self.ref_model_dir = join(self.run_data, "ref_model")
        if not os.path.exists(self.cwdir):
            os.mkdir(self.cwdir)
            os.mkdir(self.run_data)
    
        if not os.path.exists(self.run_data):
            #shutil.rmtree(self.run_data)
            os.mkdir(self.run_data)
            os.makedirs(self.image_dir, exist_ok=True)
            os.mkdir(self.ref_model_dir)

        self.curr_epoch = 0
        self.DATASET_PATH = join(self.data_set_dir,"Train/")
        self.TEST_DIR = join(self.data_set_dir,"Test/")
        self.BATCH_SIZE = 2
        self.EPOCHS = 25
        self.GEN_MODEL_PATH = join(self.cwdir,"model_gen_Conv2DLSTM_GAN.hdf5")
        self.DIS_MODEL_PATH = join(self.cwdir,"model_dis_Conv2DLSTM_GAN.hdf5")
        self.GAN_MODEL_PATH = join(self.cwdir,"model_combined_Conv2DLSTM_GAN.hdf5")
        self.REF_MODEL_PATH = join(self.cwdir, "ref_model.hdf5")

        self.penaltyLambda = 10
        self.retrain = 0
        self.curr_epoch = 1
        self.dim1 = 10
        self.dim2 = 256
        self.dim3 = 256
        self.dim4 = 1
        self.n_critic = 5
        self.r_alpha = 0.2

    def reconfig(self, new_name, batch_size = 4, epochs = 5, retrain = 0, curr_epoch = 0):
        self.cwdir_name = new_name
        self.cwdir = join(self.data_dir, self.cwdir_name)
        self.run_data = join(self.cwdir, "training_gan")
        self.image_dir = join(self.run_data,self.data_set_name,"Test/")
        self.ref_model_dir = join(self.run_data, "ref_model")

        self.BATCH_SIZE = batch_size
        self.EPOCHS = epochs
        self.GEN_MODEL_PATH = join(self.cwdir,"model_gen_Conv2DLSTM_GAN.hdf5")
        self.DIS_MODEL_PATH = join(self.cwdir,"model_dis_Conv2DLSTM_GAN.hdf5")
        self.GAN_MODEL_PATH = join(self.cwdir,"model_combined_Conv2DLSTM_GAN.hdf5")
        self.REF_MODEL_PATH = join(self.cwdir, "ref_model.hdf5")

        self.curr_epoch = 0
        self.retrain = retrain
        if retrain == 0:
            print("Configuring train from scratch")
            if not os.path.exists(self.cwdir):
                os.mkdir(self.cwdir)
                os.mkdir(self.run_data)
    
            if os.path.exists(self.run_data):
                shutil.rmtree(self.run_data)
                os.mkdir(self.run_data)
                os.makedirs(self.image_dir, exist_ok=True)
        else:
            print("Configuring retrain mode\n")
            self.final_gen_path = join(self.run_data, "generator_ep-"+str(curr_epoch)+".hdf5")
            self.final_disc_path = join(self.run_data, "discriminator_ep-"+str(curr_epoch)+".hdf5")
            self.final_gan_path = join(self.run_data, "gan_ep-"+str(curr_epoch)+".hdf5")
            self.curr_epoch = curr_epoch
            print("Last epoch saved : "+str(curr_epoch))

class ModelContainer:
    def __init__(self, y0, y1, y2, y3):
        self.generator = y0
        self.discriminator = y1
        self.G_train = y2
        self.D_train = y3
 
conf = Config(data_dir_="/content/drive/My Drive/", cwdir_name_="Conv2DLSTM_ADAE", data_set="UCSD_Anomaly_Dataset.v1p2/UCSDped1/") 
physical_devices = tf.config.list_physical_devices('GPU') 
try: 
  tf.config.experimental.set_memory_growth(physical_devices[0], True) 
except: 
  # Invalid device or cannot modify virtual devices once initialized. 
  pass

In [0]:
def get_clips_by_stride(stride, frames_list, sequence_size):

    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(sequence_size, 256, 256, 1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[cnt, :, :, 0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(clip)
                cnt = 0
    return clips

def get_clips_list(seq_size):

    clips = []
    # loop over the training folders (Train000,Train001,..)
    for f in sorted(listdir(conf.DATASET_PATH)):
        directory_path = join(conf.DATASET_PATH, f)
        if isdir(directory_path):
            all_frames = []
            # loop over all the images in the folder (0.tif,1.tif,..,199.tif)
            for c in sorted(listdir(directory_path)):
                img_path = join(directory_path, c)
                if str(img_path)[-3:] == "tif":
                    img = Image.open(img_path).resize((256, 256))
                    img = np.array(img, dtype=np.float32)
                    img = (img - 127.5)/127.5
                    #img = (2*(img - np.amin(img))/(np.amax(img) - np.amin(img)))-1
                    #img = np.array(img, dtype=np.float32) / 256.0
                    all_frames.append(img)
            # get the 32-frames sequences from the list of images after applying data augmentation
            for stride in range(1, 3):
                clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=seq_size))
    return clips


def get_single_test(single_test_path, sz):
    test = np.zeros(shape=(sz, conf.dim2, conf.dim3, conf.dim4))
    cnt = 0
    for f in sorted(listdir(single_test_path)):
        if str(join(single_test_path, f))[-3:] == "tif":
            img = Image.open(join(single_test_path, f)).resize((conf.dim2, conf.dim3))
            #cv2_imshow(np.array(img,dtype=np.float32))
            #cv2.waitKey(0)
            img = np.array(img, dtype=np.float32) / 256
            test[cnt, :, :, 0] = img
            cnt = cnt + 1
    return test

def get_test_sequences(test_case_dir, sz):
    test = get_single_test(join(conf.TEST_DIR,test_case_dir), sz)
    print("Test case loaded")
    sz = test.shape[0] - conf.dim1
    sequences = np.zeros((sz, conf.dim1, conf.dim2, conf.dim3, conf.dim4))
    # apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((conf.dim1, conf.dim2, conf.dim3, conf.dim4))
        for j in range(0, conf.dim1):
            clip[j] = test[i + j, :, :, :]
        sequences[i] = clip
    return sequences

def convert_images_back(image):
    #return np.reshape(image,(256, 256))*256.0
    #print(image.shape)
    return np.reshape(image[:, :, 0],(image.shape[0], image.shape[1]))*127.5 + 127.5

def evaluate(sequences, models, test_case_dir, gen_only):

    # get the reconstruction cost of all the sequences
    reconstructed_sequences = models.generator.predict(sequences,batch_size=conf.BATCH_SIZE)
    sr = models.discriminator.predict(reconstructed_sequences,batch_size=conf.BATCH_SIZE)

    sz = sequences.shape[0]
    
    print("Test size:"+str(sz))
    if gen_only == 1:
        os.makedirs(join(conf.image_dir,test_case_dir), exist_ok=True)
        for i in range(0, sz):
            #print("sz "+str(i)+"\n")
            #cv2_imshow(np.reshape(reconstructed_sequences[i][2],(256, 256))*256)
            if i < 10:
                img_num = "00"+str(i)
            elif i < 100:
                img_num = "0"+str(i)
            else:
                img_num = str(i)    
        
            cv2.imwrite(join(conf.image_dir, test_case_dir,"gen_"+img_num+".jpg"), convert_images_back(reconstructed_sequences[i][6]))

    plt.plot(sr)
    plt.ylabel('regularity score Sr(t)')
    plt.xlabel('frame t')
    plt.show()



In [0]:
def get_generator():
    if conf.retrain == 1 and os.path.isfile(conf.final_gen_path):
        model_gen=load_model(conf.final_gen_path, custom_objects={'LayerNormalization': LayerNormalization})
        return model_gen

    init = tf.keras.initializers.RandomNormal(stddev=0.02)
    
    print("Loading generator model")

    input_shape=(conf.dim1, conf.dim2, conf.dim3, conf.dim4)
    X_input = Input(input_shape)
    
    X = TimeDistributed(Conv2D(128, (11, 11), strides=4, kernel_initializer=init, padding="same", name="convT1"), batch_input_shape=[None, conf.dim1, conf.dim2, conf.dim3, conf.dim4])(X_input)
    X = LayerNormalization()(X)
    X = Activation("relu")(X)
    X = TimeDistributed(Conv2D(64, (5, 5), strides=2, kernel_initializer=init, padding="same", name="convT2"))(X)
    X = LayerNormalization()(X)
    X = Activation("relu")(X)
    # # # # #
    X = ConvLSTM2D(64, (3, 3), kernel_initializer=init, padding="same", return_sequences=True)(X)
    X = LayerNormalization()(X)
    X = ConvLSTM2D(32, (3, 3), kernel_initializer=init, padding="same", return_sequences=True)(X)
    X = LayerNormalization()(X)
    X = ConvLSTM2D(64, (3, 3), kernel_initializer=init, padding="same", return_sequences=True)(X)
    X = LayerNormalization()(X)
    # # # # #
    X = TimeDistributed(Conv2DTranspose(64, (5, 5), strides=2, kernel_initializer=init, padding="same", name="convT3"))(X)
    X = LayerNormalization()(X)
    X = Activation("relu")(X)
    X = TimeDistributed(Conv2DTranspose(128, (11, 11), strides=4, kernel_initializer=init, padding="same", name="convT4"))(X)
    X = LayerNormalization()(X)
    X = Activation("relu")(X)
    X = TimeDistributed(Conv2D(1, (11, 11), activation="tanh", kernel_initializer=init,  padding="same", name="convT5"))(X)

    model_gen = Model(inputs = X_input, outputs= X, name='Generator')
    model_gen.summary(line_length=150)
    return model_gen


def get_discriminator():
    if conf.retrain == 1 and os.path.isfile(conf.final_dis_path):
        model_disc=load_model(conf.final_disc_path, custom_objects={'LayerNormalization': LayerNormalization, 'ClipConstraint': ClipConstraint})
        return model_disc
    print("Loading discriminator model")

    init = tf.keras.initializers.RandomNormal(stddev=0.02)

    input_shape=[conf.BATCH_SIZE, conf.dim1, conf.dim2, conf.dim3, conf.dim4]
    X_input = Input(input_shape[1:])
    #X_input = tf.keras.layers.InputLayer(batch_size=conf.BATCH_SIZE, input_shape=input_shape)(X_input)
    
    X = TimeDistributed(Conv2D(128, (11, 11), strides=4, kernel_initializer=init, padding="same"), batch_input_shape=[None, conf.dim1, conf.dim2, conf.dim3, conf.dim4])(X_input)
    X = LayerNormalization()(X)
    X = LeakyReLU(0.2)(X)

    X = TimeDistributed(Conv2D(64, (5, 5), strides=2, kernel_initializer=init, padding="same"))(X)
    X = LayerNormalization()(X)
    X = LeakyReLU(0.2)(X)
    # # # # #
    X = TimeDistributed(Conv2D(128, (3, 3), strides=2, kernel_initializer=init, padding="same"))(X)
    X = LayerNormalization()(X)
    X = LeakyReLU(0.2)(X)

    X = TimeDistributed(Conv2D(64, (3, 3), strides=2, kernel_initializer=init, padding="same"))(X)
    X = LayerNormalization()(X)
    X = LeakyReLU(0.2)(X)

    X = TimeDistributed(Conv2D(32, (3, 3), strides=2, kernel_initializer=init, padding="same"))(X)
    X = LayerNormalization()(X)
    X = LeakyReLU(0.2)(X)

    X = Dropout(0.4)(X)
    X = Flatten()(X)
    X = Dense(512)(X)
    X = LeakyReLU(0.2)(X)
    X = Dense(1, activation='linear')(X)


    model_disc = Model(inputs = X_input, outputs = X, name='Discriminator')
    model_disc.summary(line_length=150)

    return model_disc 


def build_WGANgp(generator, discriminator):
    #### model
    # generator image(fake image)
    img_shape = [conf.dim1, conf.dim2, conf.dim3, conf.dim4]
    z = Input(shape=img_shape)
    f_img = generator(z)
    f_out = discriminator(f_img)
    # real image
    r_img = Input(shape=img_shape)
    r_out = discriminator(r_img)
    # average image
    epsilon = K.placeholder(shape=(None,1,1,1,1))
    a_img = Input(shape=(img_shape),
                  tensor = epsilon * r_img + (1-epsilon) * f_img)
    a_out = discriminator(a_img)

    #### loss
    # original critic(discriminator) loss
    r_loss = K.mean(r_out)
    f_loss = K.mean(f_out)
    # gradient penalty  <this is point of WGAN-gp>
    grad_mixed = K.gradients(a_out, [a_img])[0]
    norm_grad_mixed = K.sqrt(K.sum(K.square(grad_mixed), axis=[1,2,3,4]))
    grad_penalty = K.mean(K.square(norm_grad_mixed -1))
    penalty = conf.penaltyLambda * grad_penalty
    # d loss
    d_loss = f_loss - r_loss + penalty
    
    #### discriminator update function
    d_updates = keras.optimizers.Adam(lr=1e-4, beta_1=0.5, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([r_img, z, epsilon],
                         [r_loss, f_loss, penalty, d_loss],
                         d_updates)
    
    #### generator update function
    g_loss = -1. * f_loss
    g_updates = keras.optimizers.Adam(lr=1e-4, beta_1=0.5, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([z], [g_loss], g_updates)
    
    
    return ModelContainer(generator, discriminator, g_train, d_train)

get_generator()
get_discriminator()
#get_generator_skip_conn()
#get_discriminator_skip_conn()

In [0]:
# create a line plot of loss for the gan and save to file
def plot_history(r_loss_list, f_loss_list, f_r_loss_list, penalty_list, d_loss_list, image_name, iteration):
	# plot history
    fig, ax = plt.subplots(5, 2, figsize=(8.27,11.69))
    for j in range(2):
        ax[0,j].plot(r_loss_list, label="r_los")
        ax[1,j].plot(f_loss_list, label="f_loss")
        ax[2,j].plot(f_r_loss_list, label="f-r_loss")
        ax[3,j].plot(penalty_list, label="penalty")
        ax[4,j].plot(d_loss_list, label="d_loss")
    for i in range(5):
        ax[i,0].set_xlim([0,100])
        ax[i,1].set_xlim([100,iteration])
        for j in range(2):
            ax[i,j].grid()
            ax[i,j].legend()

    plt.legend()
    plt.savefig(image_name)
    plt.close()
 

def train_step(models, batch_clips, r_loss_list, f_loss_list, f_r_loss_list, penalty_list, d_loss_list):
  
    #### Discriminator
    for j in range(conf.n_critic):
        # Generator in
        z = batch_clips
        # Generator out Images
        f_imgs = generator.predict(z)
        # Real Images
        r_imgs = batch_clips
        # train the discriminator
        epsilon = np.random.uniform(size = (batch_size, 1,1,1,1))
        r_loss, f_loss, penalty, d_loss = D_train([r_imgs, z, epsilon])

    #### Generator
    # Generator in
    z = batch_clips
    # train the generator
    g_loss = G_train([z])

    #### Record of learning progress
    # loss
    r_loss_list.append(r_loss)
    f_loss_list.append(f_loss)
    f_r_loss_list.append(f_loss - r_loss)
    penalty_list.append(penalty)
    d_loss_list.append(d_loss)

    

    

def reconstruct_batch(model, sequences, epoch, folder_name):

    sz = sequences.shape[0]
    
    reconstructed_sequences = model.predict(sequences,batch_size=conf.BATCH_SIZE)

    path = join(conf.run_data,folder_name, str(epoch)+"_epoch")
    os.makedirs(path, exist_ok=True)
    for i in range(0, sz):
        #cv2_imshow(np.reshape(reconstructed_sequences[i][2],(256, 256))*256)
        if i < 10:
            img_num = "00"+str(i)
        elif i < 100:
            img_num = "0"+str(i)
        else:
            img_num = str(i)

        cv2.imwrite(join(path,"gen_"+img_num+".jpg"), convert_images_back(reconstructed_sequences[i][6]))
    

def train(test_cases):
    generator = get_generator()
    discriminator = get_discriminator()
    # WGAN-gp Training Model
    models = build_WGANgp(generator, discriminator)

    clips_list_arr = np.array(get_clips_list(conf.dim1))
    train_dataset = tf.data.Dataset.from_tensor_slices(clips_list_arr).batch(conf.BATCH_SIZE)

    r_loss_list, f_loss_list, f_r_loss_list, penalty_list, d_loss_list = list(), list(), list(), list(), list()
    #test_cases = get_test_sequences("Test002", 200)
    
    for epoch in range(conf.curr_epoch, conf.EPOCHS+1):
        print(f'Epoch {epoch+1} started')
        r_loss_l, f_loss_l, f_r_loss_l, penalty_l, d_loss_l = list(), list(), list(), list(), list()
        for batch in train_dataset:
            
            train_step(models, batch, r_loss_l, f_loss_l, f_r_loss_l, penalty_l, d_loss_l)
            r_loss_list.extend(r_loss_l)
            f_loss_list.extend(f_loss_l)
            f_r_loss_list.extend(f_r_loss_l)
            penalty_list.extend(penalty_l)
            d_loss_list.extend(d_loss_l)
            
        r_loss = statistics.mean(r_loss_l)
        f_loss = statistics.mean(f_loss_l)
        d_loss = statistics.mean(d_loss_l)
        print(f'Epoch: {epoch+1} \t r_loss: {r_loss} \t\t f_loss: {f_loss} \t\t d_loss: {d_loss}')

        
        models.generator.save(join(conf.run_data,"generator"+"_ep-"+str(epoch+1)+".hdf5"), save_format='h5')
        models.discriminator.save(join(conf.run_data,"discriminator"+"_ep-"+str(epoch+1)+".hdf5"), save_format='h5')

        reconstruct_batch(models.generator, test_cases[0:10*conf.BATCH_SIZE], epoch+1, "Train_Reconstructs", 1)

    
    models.generator.save(conf.GEN_MODEL_PATH,save_format='h5')
    models.discriminator.save(conf.DIS_MODEL_PATH,save_format='h5')
    plot_history(r_loss_list, f_loss_list, f_r_loss_list, penalty_list, d_loss_list, join(conf.run_data,"plot_line_plot_loss.png"), len(r_loss_list))
    return models



def get_model(new_cwdir, batch_size, epochs, test_cases, reconfig=0, reload=1, model_path=conf.GAN_MODEL_PATH, retrain=0, curr_epoch=0):
    if reconfig == 1:
        conf.reconfig(new_cwdir, batch_size, epochs, retrain, curr_epoch)
    if reload == 1 and os.path.isfile(model_path):
        print("Loading the trained model")
        model=load_model(model_path, custom_objects={'LeakyReLU': LeakyReLU, 'ClipConstraint': ClipConstraint, 'gen_loss_func': combined_loss(), 'diff_func': diff_loss(conf.BATCH_SIZE)})
        #model.load_weights(model_path)
    else :
        print("Loading the new model")
        model = train(test_cases)
    
    return model

In [0]:
#!rm training_gan/gan.hdf5 -rf
#!mkdir training_gan


test_cases_dir = "Test008"
test_cases = get_test_sequences(test_cases_dir, 200)
#model_path="/content/drive/My Drive/Conv2DLSTM_VanillaGAN/training_gan/gan_ep-2.hdf5"tf.math.divide
models = get_model(new_cwdir="Conv2DLSTM_WGAN_GP", batch_size=4, epochs=100, reload=1, test_cases=test_cases, reconfig=1)#, retrain=1, curr_epoch=39, model_path="/content/drive/My Drive/Conv2DLSTM_ADAE_ep50_b2_tanh/training_gan/gan_ep-10.hdf5")
evaluate(test_cases, models, test_cases_dir, 1)